In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def plotEEGData(EEGData, window=10, timeMin=0, timeMax=None):
    plotData = EEGData[["Fp1", "F3", "F7", "P3", "T3", "T5", "O1", "C3", "Fp2", "F4", "F8", "P4", "T4", "T6", "O2", "C4", "Fz", "Cz", "Pz"]].copy()
    
    if timeMax is None:
        timeMax = len(plotData.index)
    else:
        timeMax *= 200
    
    timeMin *= 200
    
    plotData = plotData.iloc[timeMin:timeMax]
    
    shift = 0
    plt.figure(figsize=(20, 20))
    plt.subplots_adjust(hspace=0)
    
    for idx, col in enumerate(plotData.columns):
        ax = plt.subplot(len(plotData.columns) + 2, 1, idx + 1 + shift)
        
        if idx != 0:
            ax.spines['top'].set_visible(False)
        else:
            ax.set_title("EEG Graph (Left Hemisphere, Right Hemisphere, Center)")
        
        if idx != len(plotData.columns) - 1:
            ax.spines['bottom'].set_visible(False)
        
        colData = plotData[col]
        
        graphMaxy = np.ceil(np.max(colData.rolling(window=window).mean()) / 100) * 100
        graphMiny = np.floor(np.min(colData.rolling(window=window).mean()) / 100) * 100

        graphMaxx = timeMin + int((timeMax - timeMin) * 1.005)
        graphMinx = timeMin - int((timeMax - timeMin) * 0.005)
        
        padding = np.abs(graphMaxy - graphMiny)
        
        plt.ylim(colData.min() - padding, colData.max() + padding)
        plt.yticks([graphMaxy, 0, graphMiny])
        
        plt.xlim(graphMinx, graphMaxx)
        
        if idx < len(plotData.columns) - 1:
            plt.xticks([])
        else:
            ticks1 = np.arange(np.ceil(timeMin / 1000) * 1000, np.floor(timeMax / 1000) * 1000 + 1,1000)
            ax.set_xticks(ticks1)
            ax.set_xticklabels((ticks1 / 200).astype(int))
            ax.set_xlabel("Time (sec)")

        for x in range(200, graphMaxx, 200):
            ax.axvline(x=x, ymin=colData.min() - padding, ymax=colData.max() + padding, linestyle='--', color=(0, 0, 0, 0.5))
        
        sns.lineplot(colData.rolling(window=window).mean())
        
        if idx == 7 or idx == 15:
            shift += 1
            ax = plt.subplot(len(plotData.columns) + 2, 1, idx + 1 + shift)
            
            for x in range(200, graphMaxx, 200):
                ax.axvline(x=x, ymin=colData.min() - padding, ymax=colData.max() + padding, linestyle='--', color=(0, 0, 0, 0.5))
                
            plt.ylim(colData.min() - padding, colData.max() + padding)
            plt.xlim(graphMinx, graphMaxx)
            ax.spines['bottom'].set_visible(False)
            ax.spines['top'].set_visible(False)
            plt.yticks([])
            plt.xticks([])

In [ ]:
plotEEGData(pd.read_parquet("/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1001487592.parquet"), window=10, timeMin=8, timeMax=58)